In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import math
from matplotlib.gridspec import GridSpec
plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams['font.family'] = 'Helvetica'
def rgb_to_hex(r, g, b):
    r, g, b = int(r), int(g), int(b)
    return '#' + hex(r)[2:].zfill(2) + hex(g)[2:].zfill(2) + hex(b)[2:].zfill(2)
from scipy.stats import pearsonr
from scipy.stats import spearmanr
import seaborn as sns
color_list = [rgb_to_hex(237,74,184),rgb_to_hex(158,95,253),rgb_to_hex(255,206,81),rgb_to_hex(119,200,236),rgb_to_hex(24,218,210),rgb_to_hex(111,237,155),rgb_to_hex(102,0,51),rgb_to_hex(0,0,0),rgb_to_hex(211,196,147),rgb_to_hex(186,159,72),rgb_to_hex(89,80,65),rgb_to_hex(231,85,0),rgb_to_hex(0,138,148),rgb_to_hex(223,105,98)]
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
mark=np.array(['o', 'v', '^', '<', '>', '8', 's', 'p', '*', 'h', 'H', 'D', 'd', 'P', 'X','o', 'v', '^', '<', '>', '8', 's', 'p', ])
color_list = color_list*2

In [15]:
def E(data_, h_pd=None, a=None, b=None):
	if isinstance(h_pd, pd.DataFrame): 
		eta = h_pd['h'] / data_['A']
	else: 
		eta = data_['h'] / data_['A']
	if a is None: 
		a, b = 0, 1

	E = pd.Series(0, index=data_.index)  # 각 row별로 저장할 E를 Series로 초기화

	t = [1,2,3,4,5]
	age = ['40_49', '50_59', '60_69', '70_79', '80_']

	for i in range(5):
		phi_t = np.exp(-eta / data_['eta_tilde' + age[i]])
		N_t = data_['RN' + age[i]] * data_['N']
		E += (a * t[i] + b) * N_t * phi_t  # row-wise 연산

	return E  # 각 row별 E 값이 담긴 Series 반환


def b(data_, a):
	atNp, E_0 = 0,0
	eta = data_['h']/data_['A']
	t = [1,2,3,4,5]
	age = ['40_49', '50_59', '60_69', '70_79', '80_']
	for i in range(5): 
		E_0 += sum(data_['RD'+age[i]]*data_['D'])
		phi_t = np.exp(-eta/data_['eta_tilde'+age[i]])
		N_t = data_['RN'+age[i]]*data_['N']
		atNp += sum( a*t[i]*N_t*phi_t)
	return (E_0-atNp)/E_0 # b

def find_min_max_a(year):
	# 데이터 로드
	data = pd.read_csv(f'/home/users/YongsungKwon/workplace/Yongpyter/Tuberculosis_hospital_optimization/data_result/data_over40/{year}_40.txt', sep=',')

	a_list = np.linspace(-2, 1, 31)
	a_min = None
	a_max = None

	prev_val_min = None
	prev_val_max = None

	for a_i, a in enumerate(a_list):
		# a_i 오타 수정 (ai → a_i)
		h_opt = pd.read_csv(f'/home/users/YongsungKwon/workplace/Yongpyter/Tuberculosis_hospital_optimization/data_result/at_b_tau12345/opt_E_h_a2_1/{year}MC_age_h_opt_a_{a_i}.csv')

		b_val = b(data, a)

		# 조건 1: a*5 + b crosses 0 from negative → positive
		val_min = a * 5 + b_val
		if prev_val_min is not None and prev_val_min < 0 and val_min >= 0 and a_min is None:
			a_min = a_i

		# 조건 2: a*1 + b crosses 0 from positive → negative
		val_max = a * 1 + b_val
		if prev_val_max is not None and prev_val_max >= 0 and val_max < 0:
			a_max = a_i - 1  # 직전 인덱스

		prev_val_min = val_min
		prev_val_max = val_max

	return a_min, a_max

In [29]:
a_min, a_max = find_min_max_a(2022)
print(np.linspace(-2, 1, 31)[a_min], np.linspace(-2, 1, 31)[a_max])

-1.4 0.30000000000000027


In [ ]:
phi = np.log(-data['eta'])
E = np.sum()

In [10]:
year = 2014
ai = 20# 20: 0 
a = np.linspace(-2,1,31)[ai]
data = pd.read_csv('/home/users/YongsungKwon/workplace/Yongpyter/Tuberculosis_hospital_optimization/data_result/data_over40/'+str(year)+'_40.txt',sep=',')
h_opt = pd.read_csv('/home/users/YongsungKwon/workplace/Yongpyter/Tuberculosis_hospital_optimization/data_result/at_b_tau12345/opt_E_h_a2_1/'+str(year)+'MC_age_h_opt_a_'+str(ai)+'.csv')

In [11]:
data

,sido,sigungu,h,A,N,D,RN0_9,RN10_19,RN20_29,RN30_39,...,RD40_49,RD50_59,RD60_69,RD70_79,RD80_,eta_tilde40_49,eta_tilde50_59,eta_tilde60_69,eta_tilde70_79,eta_tilde80_
0,Seoul,강남구,4,39.501308,323,12,0.002342,0.038647,0.144049,0.139657,...,0.061625,0.148459,0.128852,0.277311,0.355742,0.024210,0.028983,0.030475,0.039856,0.056153
1,Seoul,강동구,3,24.589280,288,18,0.002342,0.038647,0.144049,0.139657,...,0.061625,0.148459,0.128852,0.277311,0.355742,0.033312,0.041028,0.043533,0.060381,0.095079
2,Seoul,강북구,1,23.596920,256,16,0.002342,0.038647,0.144049,0.139657,...,0.061625,0.148459,0.128852,0.277311,0.355742,0.011571,0.014251,0.015121,0.020974,0.033026
3,Seoul,강서구,3,41.436729,266,14,0.002342,0.038647,0.144049,0.139657,...,0.061625,0.148459,0.128852,0.277311,0.355742,0.018882,0.023017,0.024340,0.033023,0.049758
4,Seoul,관악구,2,29.568727,342,14,0.002342,0.038647,0.144049,0.139657,...,0.061625,0.148459,0.128852,0.277311,0.355742,0.016555,0.019912,0.020968,0.027679,0.039640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,Gyeongnam,양산시,2,485.446929,193,6,0.001743,0.037908,0.111547,0.093682,...,0.058824,0.091503,0.058824,0.215686,0.562092,0.000957,0.001012,0.000973,0.001223,0.002041
139,Gyeongnam,진주시,3,712.957223,238,13,0.001743,0.037908,0.111547,0.093682,...,0.058824,0.091503,0.058824,0.215686,0.562092,0.001125,0.001199,0.001146,0.001500,0.002891
140,Gyeongnam,창원시,10,747.673362,702,30,0.001743,0.037908,0.111547,0.093682,...,0.058824,0.091503,0.058824,0.215686,0.562092,0.003354,0.003563,0.003414,0.004384,0.007864
141,Jeju,제주시,6,978.426995,284,11,0.008000,0.034667,0.128000,0.130667,...,0.071429,0.142857,0.285714,0.142857,0.357143,0.001446,0.001765,0.002623,0.001927,0.003494


In [12]:
h_opt

,h
0,3.66
1,3.29
2,1.54
3,3.06
4,2.42
...,...
138,1.89
139,2.96
140,8.95
141,4.46
